## How to use DNN:

1. Make sure to run the Model_Builder and Helper_functions cells to make those functions accessible
2. Do your thing in the pre-processing
3. Make sure to encode the labels vector as 0's and 1's using LabelEncoder
4. Call run_model(features, labels)

## Import all dependencies

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from keras import models, layers
import matplotlib.pyplot as plt

# Model imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Flatten, MaxPooling1D
from keras.optimizers import Adam, Nadam, RMSprop
from keras.losses import logcosh, binary_crossentropy
from keras.activations import relu, elu, sigmoid

import talos as ta
from talos.model.normalizers import lr_normalizer
from talos.model.hidden_layers import hidden_layers

# Measuring the performance of a DNN on preprocessed_ratio_data.csv

## Pre-process Data

In [ ]:
ufc_data_location = "../ufcdata/preprocessed_ratio_data.csv"

ufc_data = pd.read_csv(ufc_data_location)
ufc_data = ufc_data.drop(ufc_data.columns[0], axis=1)

In [ ]:
ufc_data

In [ ]:
"""
Retrieve all of the feature columns
"""
numerical_cols = []
categorical_cols = []
for col, col_type in zip(ufc_data.dtypes.keys(), ufc_data.dtypes):
    if col_type == "float64" or col_type == "int64":
        numerical_cols.append(col)
    else:
        categorical_cols.append(col)
        
print(len(numerical_cols))

In [ ]:
# Create the features and labels column
labels = ufc_data["Winner"]
le = LabelEncoder()
le.fit(labels)

features = ufc_data[numerical_cols].values
labels = le.transform(labels)

print("features length {}, features length {}".format(features.shape, labels.shape))

## Train model and evaluate the results

In [ ]:
results_df = run_model(features, labels)
top_10_val_acc = results_df.sort_values(by=['val_accuracy'], ascending=False).head(10)
top_10_train_acc = results_df.sort_values(by=['accuracy'], ascending=False).head(10)

In [ ]:
top_10_val_acc

In [ ]:
top_10_train_acc

## Model Builder

In [ ]:
def get_dnn(x_train, y_train, x_val, y_val, params):
    
    model = Sequential()
    # Input Layer
    model.add(Dense(params["first_neuron"], 
                    activation=params['activation'], 
                    input_dim=X_train.shape[1]))
    
    model.add(Dropout(params['dropout']))
    
    # Hidden Layers
    hidden_layers(model, params, 1)
    
    # Output Layers
    model.add(Dense(1, activation=params['last_activation']))
    
    model.compile(loss=params['losses'],
                  optimizer=params['optimizer'](lr=lr_normalizer(params['lr'], params['optimizer'])), 
                  metrics=['accuracy'])
                  
    history = model.fit(x_train, y_train,
                       validation_data=[x_val, y_val],
                       batch_size=params['batch_size'],
                       epochs=params['epochs'],
                       verbose=0)
    
    #print(model.summary())

    return history, model

In [ ]:
def run_model(x_train, y_train):
    # Define hyperparameters to use in Grid Search
    dnn_params = {
         'lr': (0.2, 1, 2),
         'first_neuron': [64, 128],
         'hidden_layers': [1, 2],
         'batch_size': [64, 128],
         'epochs': [10, 15, 25],
         'dropout': (0, 0.5, 3),
         'optimizer': [Adam],
         'shapes':['brick', 'funnel'],
         'losses': [binary_crossentropy],
         'activation': [relu],
         'last_activation': [sigmoid]}
    
    # Create the Neural Network
    dnn_model = ta.Scan(x=x_train,
               y=y_train,
               model=get_dnn,
               params=dnn_params,
               experiment_name="Winner_Predictor")
    
    return dnn_model.data

# Helper functions

In [ ]:
def plot_training_results(history_obj):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    epochs = range(1, len(acc) + 1)

    fig, (accuracy_axis, loss_axis) = plt.subplots(1, 2, figsize=(15, 5))

    accuracy_axis.plot(epochs, acc, 'bo', label='Training acc')
    accuracy_axis.plot(epochs, val_acc, 'b', label='Validation acc')
    accuracy_axis.set_title('Training and validation accuracy')
    accuracy_axis.set_ylabel('Accuracy')
    accuracy_axis.set_xlabel('Epoch')
    accuracy_axis.legend()

    loss_axis.plot(epochs, loss, 'bo', label='Training loss')
    loss_axis.plot(epochs, val_loss, 'b', label='Validation loss')
    loss_axis.set_title('Training and validation loss')
    loss_axis.set_ylim(0, 3)
    loss_axis.set_ylabel('Accuracy')
    loss_axis.set_xlabel('Epoch')
    loss_axis.legend()
    fig.show()